# Library

In [29]:
from urllib.request import urlopen
from pandas import json_normalize
from tqdm import tqdm, trange
from datetime import datetime, timedelta
from sqlalchemy import types, create_engine
from tqdm import tqdm 

import pandas as pd
import json
import time
import os
import cx_Oracle
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# Functions

In [2]:
## Get JSON from API url

def get_json(dep_city, arr_city, date, airline):
    url = (
        'https://domair.interpark.com/api/booking/airJourney.do?format=json&dep=' 
        + dep_city
        + '&arr=' 
        + arr_city
        + '&depDate=' 
        + date 
        +'&adt=1&chd=0&inf=0&tripDivi=0&airlineCode=' 
        + airline 
        + '&siteCode='
    )
    
    try:
        json_data = json.load(urlopen(url))

        #Error check
        if json_data == None:
            print("[F1]", dep_city, arr_city, date, airline, " - Null page")

        elif json_data['replyAvailFare'] == None :
            print("[F1]", dep_city, arr_city, date, airline, " - Available ticket is none")

        elif json_data['replyHeader']['errorCode'] != '0': 
            print("[F1]", dep_city, arr_city, date, airline, "Error Code: ", json_data['replyHeader']['errorCode'], " - Try again after a min ...")
            time.sleep(60)
            get_json(dep_city, arr_city, date, airline) # Recursion
            
        else:
            # print(dep_city, arr_city, date, airline,"is okay !")
            return json_data
    
    except Exception as ex:

        print("[F1] Error code: ", ex)
        print("Sleep a min ... ")

        time.sleep(60)

        print("Retry !")
        get_json(dep_city, arr_city, date, airline) # Recursion

In [3]:
## JSON parsing and get details

def get_tickets(json_data):
    df = pd.DataFrame(
        columns = ['depart_date', 'depart_time', 'airline', 'flight_no', 'depart', 'arrive', 'class_code', 'class_desc', 'avail_seat', 'fare_origin', 'fare']
    )

    try:
        for i in range(len(json_data['replyAvailFare']['availFareSet'])):
            for j in range(len(json_data['replyAvailFare']['availFareSet'][i]['segFare']['classDetail'])):
                depart_date = json_data['replyAvailFare']['availFareSet'][i]['segFare']['depDate']
                depart_time = json_data['replyAvailFare']['availFareSet'][i]['segFare']['depTime']
                airline_code = json_data['replyAvailFare']['availFareSet'][i]['segFare']['carCode']
                flight_no = json_data['replyAvailFare']['availFareSet'][i]['segFare']['mainFlt']
                depart_code = json_data['replyAvailFare']['availFareSet'][i]['segFare']['depCity']
                arrive_code = json_data['replyAvailFare']['availFareSet'][i]['segFare']['arrCity']
                class_code = json_data['replyAvailFare']['availFareSet'][i]['segFare']['classDetail'][j]['classCode']
                class_desc = json_data['replyAvailFare']['availFareSet'][i]['segFare']['classDetail'][j]['classDesc']
                avail_seat = json_data['replyAvailFare']['availFareSet'][i]['segFare']['classDetail'][j]['noOfAvailSeat']
                fare = json_data['replyAvailFare']['availFareSet'][i]['segFare']['classDetail'][j]['fare']
                fare_origin = json_data['replyAvailFare']['availFareSet'][i]['segFare']['classDetail'][j]['fareOrigin']

                data_list = [depart_date, depart_time, airline_code, flight_no, depart_code, arrive_code, class_code, class_desc, avail_seat, fare_origin, fare]
                data_series = pd.Series(data_list, index = df.columns)
                df = df.append(data_series, ignore_index = True)
                # df = pd.concat([df, data_series], ignore_index = True)

    except Exception as e:
        print("[F2] Error code: " + str(e))
            
    return df

In [5]:
## Data processing

def data_processing(ticket_df):
    
    # Search date & time adding
    ticket_df['search_date'] = datetime.today().strftime('%Y%m%d')
    ticket_df['search_time'] = datetime.today().strftime('%H%M')

    # Data types changing
    ticket_df = ticket_df.astype(
        {
            'depart_date': 'string'
            , 'depart_time': 'string'
            , 'airline': 'string'
            , 'flight_no': 'string'
            , 'depart': 'string'
            , 'arrive': 'string'
            , 'class_code': 'string'
            , 'class_desc': 'string'
            , 'avail_seat': 'int'
            , 'fare_origin': 'int'
            , 'fare': 'int'
            , 'search_date': 'string'
            , 'search_time': 'string'
        }
    )
    
    return ticket_df

In [6]:
## Stack data into Oracle DB

try:
    LOCATION = r"/Users/louienoh/instantclient10_1"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
    cx_Oracle.init_oracle_client(lib_dir="/Users/louienoh/Downloads/instantclient_19_8")
    
except Exception as er:
    print(er)

def to_oracle(df):
    id = "****"
    pw = "*****"
    address = "****"

    connect = create_engine(f"oracle+cx_oracle://{id}:{pw}@{address}")

    try:
        df.to_sql("Table name"
            , connect
            , if_exists = 'replace'
            , index = False
            , dtype={
                'depart_date': types.VARCHAR(100)
                , 'depart_time': types.VARCHAR(100)
                , 'airline': types.VARCHAR(100)
                , 'flight_no': types.VARCHAR(100)
                , 'depart': types.VARCHAR(100)
                , 'arrive': types.VARCHAR(100)
                , 'class_code': types.VARCHAR(100)
                , 'class_desc': types.VARCHAR(100)
        #         , 'avail_seat': 'int'
        #         , 'fare_origin': 'int'
                , 'fare': types.VARCHAR(100)
                , 'search_date': types.VARCHAR(100)
                , 'search_time': types.VARCHAR(100)
                 }
            )

    except Exception as ex:
        print(ex)

In [7]:
## Slack notification

def send_slack_message(channel_name, title, file_name, start_time):
    slack_token = "****"
    client = WebClient(token=slack_token)

    content = """
        %s , %s
        """ % (title, start_time)
    file_exist = False
    file_extension = ''

    if os.path.isfile(file_name):
        path, ext= os.path.splitext(file_name)
        file_extension = ext.strip('.')
        print("ext : " +file_extension)
        file_exist = True

    else:
        file_exist = False

    try:
        response = client.chat_postMessage(channel = channel_name, text = content)
        print(response['ok'])
        
        if file_exist:
            response_file_1 = client.files_upload(channels = channel_name
                                                , file = file_name
                                                , filename = file_name
                                                , filetype = file_extension
                                                )
            print(response_file_1['ok'])
        else:
            response_file_error = client.chat_postMessage(channel = channel_name, text = "No files to upload.")
            print(response_file_error['ok'])


    except SlackApiError as e:
        print('Error: {}'.format(e.response['error']))

    except Exception as e:
        print('Error : ' + str(e))

In [8]:
# ## Stack into Google Cloud Storage

# def upload_to_bucket(file_name):
#     # os.getcwd() +  
#     os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "JSON key"
#     client = storage.Client()
#     bucket = client.get_bucket("storage name")
#     blob = bucket.blob("storage name" + file_name)
#     blob.upload_from_filename(file_name)

In [9]:
# ## Create BigQuery DB

# def insert_to_bigquery_table(dataframe):
#     try:
#         os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getcwd() +  "JSON key"

#         # Construct a BigQuery client object.
#         client = bigquery.Client()

#         table_id = "****"
#         table = client.get_table(table_id)  # Make an API request.

#         job_config = bigquery.LoadJobConfig(
#             schema=[
#                 ]
#         )
        
#         job = client.load_table_from_dataframe(
#             dataframe, table_id, job_config=job_config
#         )  # Make an API request.

#         job.result()  
#         # Wait for the job to complete.

#         table = client.get_table(table_id)  # Make an API request.
#         print(
#             "Loaded {} rows and {} columns to {}".format(
#                 table.num_rows, len(table.schema), table_id
#             )
#         )

#     except Exception as e:
#         print("Error : " + str(e))

# Main

In [10]:
start_time = datetime.now()
print("Start: ", start_time.strftime('%Y-%m-%d %H:%M'))

Start:  2023-01-09 16:49


In [ ]:
# send_slack_message("crawling_logs", "airline_ticket (시작)", "", start_time)

In [12]:
airport_5 = ['SEL', 'PUS', 'CJJ', 'KWJ', 'TAE']

air_route = []

for depart in airport_5:
    # for arrive in jeju:
    air_route.append([depart, 'CJU'])
    air_route.append(['CJU', depart])

air_route

[['SEL', 'CJU'],
 ['CJU', 'SEL'],
 ['PUS', 'CJU'],
 ['CJU', 'PUS'],
 ['CJJ', 'CJU'],
 ['CJU', 'CJJ'],
 ['KWJ', 'CJU'],
 ['CJU', 'KWJ'],
 ['TAE', 'CJU'],
 ['CJU', 'TAE']]

In [13]:
airline_list = ['KE', 'OZ', '7C', 'LJ', 'TW', 'BX', 'RS']
airline_list

['KE', 'OZ', '7C', 'LJ', 'TW', 'BX', 'RS']

In [15]:
final_df = pd.DataFrame()
    
for date in trange(1, 4):
    
    date = datetime.today() + timedelta(days = date)
    date = date.strftime('%Y%m%d')
    # print(date)
    
    for airline in airline_list:

        for line in air_route:

            json_data = get_json(line[0], line[1], date, airline)
            
            if json_data == None:
                
                continue

            else:

                ticket_df = get_tickets(json_data)

                total_df = data_processing(ticket_df)
                
                final_df = final_df.append(total_df, ignore_index = True)
                
to_oracle(final_df)

                # # to parquet
                # end_time = datetime.now()
                # e_time_re = end_time.strftime('%Y%m%d_%H%M')
                # filename = "airline_ticket_" + e_time_re + ".parquet"
                # total_df.to_parquet(filename, engine="pyarrow", compression="gzip")
                # upload_to_bucket(filename)
                # insert_to_bigquery_table(total_df)

  0%|          | 0/3 [00:00<?, ?it/s]

[F1] TAE CJU 20230110 KE  - Available ticket is none
[F1] PUS CJU 20230110 OZ  - Available ticket is none
[F1] CJU PUS 20230110 OZ  - Available ticket is none
[F1] SEL CJU 20230110 7C  - Available ticket is none
[F1] CJU SEL 20230110 7C  - Available ticket is none
[F1] KWJ CJU 20230110 7C  - Available ticket is none
[F1] SEL CJU 20230110 LJ  - Available ticket is none
[F1] CJU SEL 20230110 LJ  - Available ticket is none
[F1] SEL CJU 20230110 TW  - Available ticket is none
[F1] CJU SEL 20230110 TW  - Available ticket is none
[F1] PUS CJU 20230110 TW  - Available ticket is none
[F1] CJU PUS 20230110 TW  - Available ticket is none
[F1] SEL CJU 20230110 BX  - Available ticket is none
[F1] CJU SEL 20230110 BX  - Available ticket is none
[F1] CJJ CJU 20230110 BX  - Available ticket is none
[F1] CJU CJJ 20230110 BX  - Available ticket is none
[F1] KWJ CJU 20230110 BX  - Available ticket is none
[F1] CJU KWJ 20230110 BX  - Available ticket is none
[F1] TAE CJU 20230110 BX  - Available ticket i

 33%|███▎      | 1/3 [00:30<01:01, 30.72s/it]

[F1] CJU TAE 20230110 RS  - Available ticket is none
[F1] PUS CJU 20230111 OZ  - Available ticket is none
[F1] SEL CJU 20230111 7C  - Available ticket is none
[F1] CJU SEL 20230111 7C  - Available ticket is none
[F1] SEL CJU 20230111 LJ  - Available ticket is none
[F1] CJU SEL 20230111 LJ  - Available ticket is none
[F1] SEL CJU 20230111 TW  - Available ticket is none
[F1] CJU SEL 20230111 TW  - Available ticket is none
[F1] PUS CJU 20230111 TW  - Available ticket is none
[F1] CJU PUS 20230111 TW  - Available ticket is none
[F1] SEL CJU 20230111 BX  - Available ticket is none
[F1] CJU SEL 20230111 BX  - Available ticket is none
[F1] CJJ CJU 20230111 BX  - Available ticket is none
[F1] CJU CJJ 20230111 BX  - Available ticket is none
[F1] KWJ CJU 20230111 BX  - Available ticket is none
[F1] CJU KWJ 20230111 BX  - Available ticket is none
[F1] TAE CJU 20230111 BX  - Available ticket is none
[F1] CJU TAE 20230111 BX  - Available ticket is none
[F1] SEL CJU 20230111 RS  - Available ticket i

 67%|██████▋   | 2/3 [01:12<00:36, 36.94s/it]

[F1] CJU TAE 20230111 RS  - Available ticket is none
[F1] CJU PUS 20230112 OZ  - Available ticket is none
[F1] SEL CJU 20230112 7C  - Available ticket is none
[F1] CJU SEL 20230112 7C  - Available ticket is none
[F1] SEL CJU 20230112 LJ  - Available ticket is none
[F1] CJU SEL 20230112 LJ  - Available ticket is none
[F1] SEL CJU 20230112 TW  - Available ticket is none
[F1] CJU SEL 20230112 TW  - Available ticket is none
[F1] PUS CJU 20230112 TW  - Available ticket is none
[F1] CJU PUS 20230112 TW  - Available ticket is none
[F2] Error code: object of type 'NoneType' has no len()
[F1] SEL CJU 20230112 BX  - Available ticket is none
[F1] CJU SEL 20230112 BX  - Available ticket is none
[F1] CJJ CJU 20230112 BX  - Available ticket is none
[F1] CJU CJJ 20230112 BX  - Available ticket is none
[F1] KWJ CJU 20230112 BX  - Available ticket is none
[F1] CJU KWJ 20230112 BX  - Available ticket is none
[F1] TAE CJU 20230112 BX  - Available ticket is none
[F1] CJU TAE 20230112 BX  - Available ticke

100%|██████████| 3/3 [01:50<00:00, 36.94s/it]

[F1] CJU TAE 20230112 RS  - Available ticket is none


In [28]:
final_df.query("depart_date == '20230112' and arrive == 'GMP'").sort_values("fare", ascending = True)

,depart_date,depart_time,airline,flight_no,depart,arrive,class_code,class_desc,avail_seat,fare_origin,fare,search_date,search_time
1116,20230112,0700,OZ,8902,CJU,GMP,G,특가석,1,26000,26000,20230109,1659
989,20230112,0700,KE,1118,CJU,GMP,V,특가석,4,49300,31000,20230109,1659
1120,20230112,0800,OZ,8904,CJU,GMP,T,특가석,1,31000,31000,20230109,1659
1131,20230112,1055,OZ,8934,CJU,GMP,W,특가석,2,35000,35000,20230109,1659
992,20230112,0925,KE,1158,CJU,GMP,U,특가석,1,56300,38000,20230109,1659
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,20230112,1400,OZ,8950,CJU,GMP,C,비즈니스석,6,146000,146000,20230109,1659
1156,20230112,1540,OZ,8958,CJU,GMP,C,비즈니스석,3,146000,146000,20230109,1659
983,20230112,1700,KE,1278,CJU,GMP,C,비즈니스석,1,164300,146000,20230109,1659
1140,20230112,1250,OZ,8940,CJU,GMP,C,비즈니스석,1,146000,146000,20230109,1659


In [16]:
end_time = datetime.now()
running_time = end_time - start_time
print("End: ", end_time.strftime('%Y-%m-%d %H:%M'))
print("Running time: ", running_time)

# send_slack_message("crawling_logs", "airline_ticket (종료)", "airline.log", running_time)

End:  2023-01-09 17:01
Running time:  0:12:32.488065
